In [2]:
library(rhdf5)
library(Matrix)
library(SingleCellExperiment)

In [2]:
mydir = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/in_vivo_comparison/"

In [3]:
filename = "/hps/nobackup/stegle/users/acuomo/all_scripts/sc_neuroseq/scanpy/5pct_subsample.h5ad"

In [4]:
cbind(h5ls(filename)$group,
h5ls(filename)$name)

/,X
/X,data
/X,indices
/X,indptr
/,obs
/obs,__categories
/obs/__categories,batch
/obs/__categories,celltype
/obs/__categories,donor_id
/obs/__categories,pool_id
/obs/__categories,sample_id


In [5]:
# unique(h5read(file = filename, "obs")["celltype"])

In [6]:
# h5read(file = filename, "uns")$celltype_categories

In [7]:
ind = h5read(file = filename, "obs")$index # cell names
var = h5read(file = filename, "var") # gene names

In [8]:
head(ind)

[1] "ATCGAGTTCAAACGGG-1-60"  "ACCAGTAAGACCTAGG-1-81"  "GCTGCAGCAACCGCCA-1-46" 
[4] "AACTCCCTCGACAGCC-1-66"  "CGTCCATGTAGGCTGA-1-53"  "CTCTACGTCGCATGAT-1-129"

In [9]:
var$index[grep("NANOG",var$index)]

[1] "NANOGNB" "NANOG"

In [10]:
X = h5read(file = filename, "X")
X_data = as.integer(X$data)
X_indptr = as.integer(X$indptr)
X_indices = as.integer(X$indices)
rm(X)

In [11]:
sMat = sparseMatrix(
i = X_indices,
p = X_indptr,
x = X_data,
dims = c(length(var[[1]]), length(ind)),
index1 = FALSE
)
rownames(sMat) <- as.character(var[[1]])
colnames(sMat) <- as.character(ind)

In [12]:
rownames(sMat)[grep("SOX",rownames(sMat))]

[1] "QSOX1"     "SOX13"     "SOX11"     "SOX14"     "SOX2"      "SOX30"    
 [7] "SOX4"      "SOX3"      "SOX7"      "SOX7-1"    "SOX17"     "QSOX2"    
[13] "SOX6"      "SOX5"      "SOX21"     "SOX21-AS1" "SOX1"      "SOX8"     
[19] "SOX15"     "SOX9"      "SOX12"     "SOX18"     "SOX10"

In [13]:
R_filename = paste0(mydir,"neuroseq_all_5pct.Rds")
# saveRDS(sMat, file = R_filename)

In [14]:
obs = h5read(file = filename, "obs")
str(obs)

List of 22
 $ __categories               :List of 7
  ..$ batch     : chr [1:151(1d)] "0" "1" "2" "3" ...
  ..$ celltype  : chr [1:11(1d)] "Astro" "CHem" "DA" "Epen1" ...
  ..$ donor_id  : chr [1:213(1d)] "HPSI0114i-bezi_3" "HPSI0114i-eipl_1" "HPSI0114i-fikt_3" "HPSI0114i-kolf_2" ...
  ..$ pool_id   : chr [1:17(1d)] "pool1" "pool2" "pool3" "pool4" ...
  ..$ sample_id : chr [1:151(1d)] "5245STDY7352549" "5245STDY7352550" "5245STDY7386985" "5245STDY7386986" ...
  ..$ time_point: chr [1:3(1d)] "D11" "D30" "D52"
  ..$ treatment : chr [1:2(1d)] "NONE" "ROT"
 $ batch                      : int [1:205416(1d)] 60 81 46 66 53 129 134 79 37 119 ...
 $ celltype                   : int [1:205416(1d)] 2 0 2 7 5 2 9 2 5 9 ...
 $ cluster_id                 : int [1:205416(1d)] 1 0 1 5 4 1 2 1 3 2 ...
 $ donor_id                   : int [1:205416(1d)] 87 11 40 66 23 86 103 66 170 49 ...
 $ index                      : chr [1:205416(1d)] "ATCGAGTTCAAACGGG-1-60" "ACCAGTAAGACCTAGG-1-81" "GCTGCAGCAACCGCCA

In [44]:
df = data.frame(row.names = obs$index)
head(df)

ATCGAGTTCAAACGGG-1-60
ACCAGTAAGACCTAGG-1-81
GCTGCAGCAACCGCCA-1-46
AACTCCCTCGACAGCC-1-66
CGTCCATGTAGGCTGA-1-53
CTCTACGTCGCATGAT-1-129


In [45]:
obs = h5read(file = filename, "obs")
all_cols = names(obs)
categorical_cols = names(h5read(file = filename, "obs/__categories"))

In [52]:
for (col in all_cols){
#     print(col)
    if (!(col %in% c(categorical_cols, "__categories","index"))){
#         print(col)
#         print(obs[[col]])
        df[[col]] = (as.numeric(obs[[col]]))   
    }
    if (col %in% c(categorical_cols)){
        colname = paste0("obs/__categories/",col)
        df[[col]] = h5read(file = filename,colname)[((as.numeric(obs[[col]]))+1)]
    }
}

In [53]:
head(df)

,cluster_id,log1p_n_genes_by_counts,log1p_total_counts,log1p_total_counts_mito,n_counts_all,n_genes_by_counts,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,pct_counts_in_top_50_genes,pct_counts_mito,total_counts,total_counts_mito,batch,celltype,donor_id,pool_id,sample_id,time_point,treatment
ATCGAGTTCAAACGGG-1-60,1,8.347827,9.487972,5.998937,13199,4220,29.72195,38.36654,53.48890,22.88052,3.0456853,13199,402,60,DA,HPSI0514i-fiaj_1,pool5,5245STDY7631342,D52,NONE
ACCAGTAAGACCTAGG-1-81,0,8.043984,9.088625,5.181784,8853,3114,34.74528,45.88275,60.74777,24.08223,1.9993223,8853,177,81,Astro,HPSI0115i-aoxv_1,pool8,5245STDY7654676,D30,NONE
GCTGCAGCAACCGCCA-1-46,1,6.579251,7.176255,2.890372,1307,719,44.07039,59.83168,83.24407,31.75210,1.3006885,1307,17,46,DA,HPSI0214i-poih_2,pool5,5245STDY7603455,D30,NONE
AACTCCCTCGACAGCC-1-66,5,7.980024,9.723763,5.087596,16709,2921,58.68694,68.40625,78.79586,42.61775,0.9635526,16709,161,66,P_FPP,HPSI0414i-mita_1,pool6,5245STDY7632692,D11,NONE
CGTCCATGTAGGCTGA-1-53,4,7.807103,9.137662,4.969813,9298,2457,52.64573,61.91654,74.16649,38.13723,1.5379652,9298,143,53,FPP,HPSI0115i-paim_1,pool1,5245STDY7631335,D52,NONE
CTCTACGTCGCATGAT-1-129,1,8.450840,9.791606,6.368187,17882,4678,31.88122,41.53898,56.85606,24.44357,3.2546694,17882,582,129,DA,HPSI0513i-veve_2,pool11,5245STDY7899446,D30,NONE


In [58]:
cData_raw = df
head(cData_raw)

,cluster_id,log1p_n_genes_by_counts,log1p_total_counts,log1p_total_counts_mito,n_counts_all,n_genes_by_counts,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,pct_counts_in_top_50_genes,pct_counts_mito,total_counts,total_counts_mito,batch,celltype,donor_id,pool_id,sample_id,time_point,treatment
ATCGAGTTCAAACGGG-1-60,1,8.347827,9.487972,5.998937,13199,4220,29.72195,38.36654,53.48890,22.88052,3.0456853,13199,402,60,DA,HPSI0514i-fiaj_1,pool5,5245STDY7631342,D52,NONE
ACCAGTAAGACCTAGG-1-81,0,8.043984,9.088625,5.181784,8853,3114,34.74528,45.88275,60.74777,24.08223,1.9993223,8853,177,81,Astro,HPSI0115i-aoxv_1,pool8,5245STDY7654676,D30,NONE
GCTGCAGCAACCGCCA-1-46,1,6.579251,7.176255,2.890372,1307,719,44.07039,59.83168,83.24407,31.75210,1.3006885,1307,17,46,DA,HPSI0214i-poih_2,pool5,5245STDY7603455,D30,NONE
AACTCCCTCGACAGCC-1-66,5,7.980024,9.723763,5.087596,16709,2921,58.68694,68.40625,78.79586,42.61775,0.9635526,16709,161,66,P_FPP,HPSI0414i-mita_1,pool6,5245STDY7632692,D11,NONE
CGTCCATGTAGGCTGA-1-53,4,7.807103,9.137662,4.969813,9298,2457,52.64573,61.91654,74.16649,38.13723,1.5379652,9298,143,53,FPP,HPSI0115i-paim_1,pool1,5245STDY7631335,D52,NONE
CTCTACGTCGCATGAT-1-129,1,8.450840,9.791606,6.368187,17882,4678,31.88122,41.53898,56.85606,24.44357,3.2546694,17882,582,129,DA,HPSI0513i-veve_2,pool11,5245STDY7899446,D30,NONE


In [59]:
cData_raw$umap1 = h5read(file = filename, "obsm")$X_umap[1,]
cData_raw$umap2 = h5read(file = filename, "obsm")$X_umap[2,]

In [60]:
cData_raw$index = rownames(cData_raw)

In [61]:
head(cData_raw)

,cluster_id,log1p_n_genes_by_counts,log1p_total_counts,log1p_total_counts_mito,n_counts_all,n_genes_by_counts,pct_counts_in_top_100_genes,pct_counts_in_top_200_genes,pct_counts_in_top_500_genes,pct_counts_in_top_50_genes,...,batch,celltype,donor_id,pool_id,sample_id,time_point,treatment,umap1,umap2,index
ATCGAGTTCAAACGGG-1-60,1,8.347827,9.487972,5.998937,13199,4220,29.72195,38.36654,53.48890,22.88052,...,60,DA,HPSI0514i-fiaj_1,pool5,5245STDY7631342,D52,NONE,5.426174,9.667524,ATCGAGTTCAAACGGG-1-60
ACCAGTAAGACCTAGG-1-81,0,8.043984,9.088625,5.181784,8853,3114,34.74528,45.88275,60.74777,24.08223,...,81,Astro,HPSI0115i-aoxv_1,pool8,5245STDY7654676,D30,NONE,-3.218357,-4.263032,ACCAGTAAGACCTAGG-1-81
GCTGCAGCAACCGCCA-1-46,1,6.579251,7.176255,2.890372,1307,719,44.07039,59.83168,83.24407,31.75210,...,46,DA,HPSI0214i-poih_2,pool5,5245STDY7603455,D30,NONE,6.085959,6.382314,GCTGCAGCAACCGCCA-1-46
AACTCCCTCGACAGCC-1-66,5,7.980024,9.723763,5.087596,16709,2921,58.68694,68.40625,78.79586,42.61775,...,66,P_FPP,HPSI0414i-mita_1,pool6,5245STDY7632692,D11,NONE,-6.902784,2.802364,AACTCCCTCGACAGCC-1-66
CGTCCATGTAGGCTGA-1-53,4,7.807103,9.137662,4.969813,9298,2457,52.64573,61.91654,74.16649,38.13723,...,53,FPP,HPSI0115i-paim_1,pool1,5245STDY7631335,D52,NONE,-4.819849,-8.872992,CGTCCATGTAGGCTGA-1-53
CTCTACGTCGCATGAT-1-129,1,8.450840,9.791606,6.368187,17882,4678,31.88122,41.53898,56.85606,24.44357,...,129,DA,HPSI0513i-veve_2,pool11,5245STDY7899446,D30,NONE,6.874391,7.509790,CTCTACGTCGCATGAT-1-129


In [62]:
cData <- as.data.frame(cData_raw)
cData_R_filename = paste0(mydir,"neuroseq_all_5pct_cData.Rds")
saveRDS(cData, file = cData_R_filename)

In [63]:
# build the sce object
sce <- SingleCellExperiment(
assays = list(counts = sMat),
colData = cData)
sce

class: SingleCellExperiment 
dim: 32738 205416 
metadata(0):
assays(1): counts
rownames(32738): MIR1302-10 FAM138A ... AC002321.2 AC002321.1
rowData names(0):
colnames(205416): ATCGAGTTCAAACGGG-1-60 ACCAGTAAGACCTAGG-1-81 ...
  ACTTTCAAGACCGGAT-1-26 TACACGAAGTGTCCCG-1-145
colData names(23): cluster_id log1p_n_genes_by_counts ... umap2 index
reducedDimNames(0):
spikeNames(0):
altExpNames(0):

In [64]:
sce_R_filename = paste0(mydir,"neuroseq_all_5pct_sce.Rds")
saveRDS(sce, file = sce_R_filename)

In [65]:
rownames(sce)[grep("SOX",rownames(sce))]

[1] "QSOX1"     "SOX13"     "SOX11"     "SOX14"     "SOX2"      "SOX30"    
 [7] "SOX4"      "SOX3"      "SOX7"      "SOX7-1"    "SOX17"     "QSOX2"    
[13] "SOX6"      "SOX5"      "SOX21"     "SOX21-AS1" "SOX1"      "SOX8"     
[19] "SOX15"     "SOX9"      "SOX12"     "SOX18"     "SOX10"